# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

In [19]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

URL = 'https://habr.com/ru/all/'
res = requests.get(URL)

KEYWORDS = ['python', 'работа', 'игры']

soup = BeautifulSoup(res.text, 'html.parser')

data_dict = {
    'date': [],
    'title': [],
    'link': [],
}

In [20]:
def get_url_data(url, page):
    if page != 1:
        url = url + '/page' + str(page)
    res = requests.get(url)
    time.sleep(0.3)
    soup = BeautifulSoup(res.text, 'html.parser')
    posts = soup.find_all('article', class_='post post_preview')
    for post in posts:
        hubs = post.find_all('a', class_='post__title_link')
        post_time = post.find('span', class_='post__time').text
        for hub in hubs:
            hub_lower = hub.text.lower()
            for keyword in KEYWORDS:
                if keyword in hub_lower:
                    title_element = post.find('a', class_='post__title_link')
                    return post_time, title_element.text, title_element.attrs.get('href')

In [22]:
for page in range(1,10):
    if get_url_data(URL, page):
        print(f'Нашлось на станице новостей №{page}')
        date, title, link = get_url_data(URL, page)
        data_dict['date'].append(date)
        data_dict['title'].append(title)
        data_dict['link'].append(link)

Нашлось на станице новостей №1
Нашлось на станице новостей №2
Нашлось на станице новостей №4
Нашлось на станице новостей №5


In [23]:
filtered_news = pd.DataFrame(data_dict)
filtered_news

,date,title,link
0,сегодня в 18:44,"Создание арт-объектов, игр и много чего ещё с ...",https://habr.com/ru/company/skillfactory/blog/...
1,сегодня в 14:58,Как мы визуальный конструктор обучения цифровы...,https://habr.com/ru/company/neuronet/blog/548934/
2,сегодня в 10:01,Как этот год работала наша внутренняя служба г...,https://habr.com/ru/company/croc/blog/548768/
3,вчера в 18:00,Работаем с lightsquid или как сделать индивиду...,https://habr.com/ru/post/548782/
4,сегодня в 18:44,"Создание арт-объектов, игр и много чего ещё с ...",https://habr.com/ru/company/skillfactory/blog/...
5,сегодня в 14:58,Как мы визуальный конструктор обучения цифровы...,https://habr.com/ru/company/neuronet/blog/548934/
6,сегодня в 10:01,Как этот год работала наша внутренняя служба г...,https://habr.com/ru/company/croc/blog/548768/
7,вчера в 18:00,Работаем с lightsquid или как сделать индивиду...,https://habr.com/ru/post/548782/


## Задание 2.

In [25]:
import json

params = {
        'emailAddresses': ["yy@ya.ru","xx@ya.ru", "12@wwwww.ru"]
    }

URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

headers = {
    'Host': 'identityprotection.avast.com',
    'Connection': 'keep-alive',
    'Content-Length': '34',
    'sec-ch-ua': '"Google Chrome";v="87", " Not;A Brand";v="99", "Chromium";v="87"',
    'Accept': 'application/json, text/plain, */*',
    'Vaar-Version': '0',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Content-Type': 'application/json;charset=UTF-8',
    'Origin': 'https://www.avast.com',
    'Sec-Fetch-Site': 'same-site',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': 'https://www.avast.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7'
}



res = requests.post(URL, headers=headers, json=params)
res

<Response [400]>

<Response [200]>